# GA Python

Here is a simple example of using a GA in Python using PyOptSparse.  PyOptSparse uses input and output dictionaries for convenience when dealing with many different types of variables and constraints.  It's a bit inconvenient for simple functions like this, but useful for dealing with complex functions with many variables.

In [2]:
def rosen(xdict):
    x = xdict['x']

    f = (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

    funcs = {}
    funcs['obj'] = f
    funcs['con'] = []
    fail = False

    return funcs, fail

Here is where we define the problem and choose an optimizer.  Various options exist, I've set a few, see documentation for details.

In [6]:
from pyoptsparse import Optimization, NSGA2

# setup problem
dim = 2
opt_prob = Optimization('rosenbrock', rosen)
opt_prob.addObj('obj')  # use same name as before
opt_prob.addVarGroup('x', dim, type='c', lower=-5.0, upper=5.0, value=4.0)  # value not used by NSGA

# choose optimizer and define options
nsga_opt = NSGA2()
nsga_opt.setOption('maxGen', 100*dim)
nsga_opt.setOption('PopSize', 20*dim)
nsga_opt.setOption('pMut_real', 0.01)
nsga_opt.setOption('pCross_real', 1.0)

Now we can run the optimizer and parse the results.

In [4]:
sol = nsga_opt(opt_prob)

xstar = sol.xStar['x']
fstar, _ = rosen(sol.xStar)
fstar = fstar['obj']
print xstar, fstar

[ 0.80973481  0.65449819] 0.0363382656261


NSGA doesn't have any convergence criteria, other than the maximum number of generations.  I set it at 200 in this case.  Notice that we have a pretty good answer, but convergence is no very tight.

Let's also try with SNOPT and start fairly far away (and I won't supply gradients):

In [8]:
from pyoptsparse import SNOPT

snopt = SNOPT()

sol = snopt(opt_prob)
xstar = sol.xStar['x']
fstar, _ = rosen(sol.xStar)
fstar = fstar['obj']
print xstar, fstar

[ 1.00000076  1.00000152] 5.8876451256e-13


We have the answer to high precision, and its fast and repeatable.  For something that is differentiable a gradient-based method is preferable, but if the function space is fundamentally noisy, discrete, or highly multi-modal then a GA or other gradient-free method can be effective.